# Probe design workflow
## Inputs for DesignTargets
1. Reference fasta file
    - Sequence names must be the be simple name
    - Must contain all the sequences one wishes to screen against off target effects
2. (optional) List of sequence names that are not expressed

## Inputs for GeneExpression
1. Expression FPKM or counts for different genes (must have same names as used in DesignTargets
2. GTF file

# Dependencies
1. OligoArray2.jar
2. formatdb (legacy)
3. BLAST (legacy)
4. Bio python package
5. pandas

In [ ]:
"""
0.2 Update Changes

Change parse oligo to allow user to specify path in order to reload already completed stuffs.
"""

In [8]:
from Bio import SeqIO
from tempfile import mkdtemp
from os import rmdir, path, mkdir, system
from shutil import rmtree
from subprocess import call
import uuid
from pandas import DataFrame


# Path to formatdb, BLAST, and any other dependent tools


In [16]:
class DesignTargets():
    """
    Class designed to generate hybridization probes against target mRNA while 
    minimizing the offtarget binding of the probe.
    Inputs:
    Ensemble ID of targets and a fasta of all off target references to screen against.
    OligoArray2.1
    Old BLAST
    """
    def __init__(self,
                 transcript_fasta = '/data/v24.transcripts.trimmed.fa', 
                 temp_dir = '/data/'):
        """
        Initiliaze the class with a fasta file of offtarget sequences.
        A root directory to store temporary files.
        Some parameters of the probe selection algorithms.
        """
        self.jarpth = '/data/wollman/tools/bin/OligoArray2.jar'
        self._assignReferences(transcript_fasta)
        self._temp_dir = path.join(temp_dir, str(uuid.uuid4()))
        mkdir(self._temp_dir)
        self.gc_min = 20
        self.gc_max = 80
        self.length = 30
        self.min_t = 70
        self.max_t = 100
        self.xhybe = 72
        self.tsecondary = 76
        self.nthreads = 16
        
    def designProbes(self, transcript_id = None, not_expressed = None):
        # Type checking inputs and property states
        if transcript_id is None:
            if self._targetTranscripts is None:
                raise Exception('No known transcripts to design probes for.')
            else:
                transcript_id = self._targetTranscripts
        elif not isinstance(transcript_id, list):
            if isinstance(transcript_id, str):
                transcript_id = [transcript_id]
        if not_expressed is None:
            blastdb = self._makeblastdb(fdir = path.join(self._temp_dir, 'db.fa'))
        else:
            trimmed_records = list(set(self._records.keys()) - set(not_expressed))
            blastdb = self._makeblastdb(records=trimmed_records, fdir = path.join(self._temp_dir, 'db.fa'))
        # Iterate over the transcripts that need targets designed
        self._oligoarray2(transcript_id, blastdb=blastdb)
                
    def _oligoarray2(self, sequences, blastdb = None):
        """
        Wrapper for OligoArray2.1 system call.
        Given a list of sequence names run the oligoarray2.1 jar file.
        Use the parameters as they are currently in the instance.
        """
        fname = self._writefasta(sequences, fdir = path.join(self._temp_dir, 'temp.fa'))
        oligos_out = path.join(self._temp_dir, 'oligos.txt')
        log_out = path.join(self._temp_dir, 'oligos.log')
        !java -Xmx4g -jar {self.jarpth} -i {fname} -d {blastdb} -o {oligos_out} -R {log_out} -n 196 -l 30 -L 30 -D 10000 -t {self.min_t} -T {self.max_t} -s {self.tsecondary} -x {self.xhybe} -N {self.nthreads} -g 30 -m "GGGGGG;CCCCCC;TTTTTT;AAAAAA" -p {self.gc_min} -P {self.gc_max}

    def _assignReferences(self, fastafile):
        """
        Assecory function to setup fasta files.
        """
        if isinstance(fastafile, str):
            self._records = SeqIO.index(fastafile, 'fasta')
        elif isinstance(fastafile, SeqIO.index):
            self._records = fastafile
    def _writefasta(self, transcript_ids, temp=True, fdir = 'tempfile.fa'):
        """
        Assecory function to write a fasta file containing the subset of 
        intended target sequence names.
        """
        sequences = [self._records.get(seq) for seq in transcript_ids]
        print(sequences)
        with open(fdir, 'w') as handle:
            SeqIO.write(sequences, handle, 'fasta')
        return fdir
    def _makeblastdb(self, records=None, fdir = 'tempdb.fa'):
        """
        Transform the fasta file of possible off-target sequences 
        into a blastdb indexed format necessary for OligoArray2.1.
        """
        if records is None:
            records = self._records
            with open(fdir, 'w') as handle:
                SeqIO.write(records.values(), handle, 'fasta')
        if isinstance(records, list):
            with open(fdir, 'w') as handle:
                SeqIO.write([self._records.get(i) for i in records], handle, 'fasta')
        !formatdb -i {fdir} -o T -p F
        return fdir
    
    def _parseOligoArrayOutput(self, fname='oligos.txt', fpth=None):
        """
        Parse the output file into two dataframe objects.
        One contains the parameters of the hybridixation between candidate probes 
        and their cognate mRNA. Another contains all the possible offtarget binding 
        sites between candidate probes and sequences in the off-target database.
        The designer writes a CSV that this parses.
        """
        probe_colnames = ['target', 'pos', 'len', 'dG', 'dH', 'dS', 'Tm', 'seq', 'off_df']
        probes = []
        offtargets = []
        if fpth is not None:
            filename = fpth
        else:
            filename = path.join(self._temp_dir, fname)
        with filename as f:
            for line in f.readlines():
                line = line.rstrip()
                columns = line.split('\t')
                seq = columns[-1]
                probe_target = columns[0]
                pos = columns[1]
                offt = columns[-2].split(';')
                parent = offt[0].strip()
                # Edge case in the output formatting
                if parent != probe_target:
                    parent_split = parent.find(',')
                    offt.append(parent[parent_split+1:])
                    parent = probe_target
                    print('Warning', parent, probe_target)
                offtarg_sites = []
                for off_site in offt[1:]:
                    names, binding = off_site.split('(')
                    names = names.strip()
                    names = names.split(',')
                    names = [n.strip() for n in names]
                    binding = binding.strip(')').split(' ')
                    for n in names:
                        offtarg_sites.append([parent, pos, n]+binding)
                probes.append(columns[:-2]+[seq]+
                              [DataFrame(offtarg_sites, 
                                         columns=['target', 'pos', 'offtarget', 
                                                  'dG', 'dH', 'dS', 'tm', 'offseq'])
                              ])
                
        probe_df = DataFrame(probes, columns=probe_colnames)
        return probe_df
    
    def cleanup(self):
        rmtree(self._temp_dir)

In [ ]:
class MerprobeOligos():
    
    def __init__(primer_path = '/data/wollman/hypefish/datafiles/PrimerSeqs.fasta'):
        # Create a dictionary of possible primers each time it generates a primer pair 
        # they will be removed from the dict to prevent reuse
        primer_seqs = {}
        fasta = SeqIO.parse(primer_path, 'fasta')
        for seq in fasta:
            primer_seqs[seq.id] = str(seq.seq)
        fasta.close()

In [8]:
from collections import Counter

class ScreenTargets():
    def __init__(self, df, arg=None):
        self.probedf = df
    def _parseProbeQuality(self, input_df, arg = None):
        """
        Make a df with columns for each candidate probe 
        that ranks how good and specific it is.
        """
        if arg is None:
            arg = {'output': ['sum_exp', 'numoff', ]}
    def addQualityColumns(self, exp_df = None):
        """
        Notes: exp_df must have a column transcript_id and SUM_FPKM.
        """
        self.probedf['numoff'] = [row.off_df.shape[0] for ix, row in self.probedf.iterrows()]
        if exp_df is not None:
            exp_values = []
            for ix, row in self.probedf.iterrows():
                ids = row.off_df.offtarget.values
                exp_levels = exp_df['SUM_FPKM'][exp_df['transcript_id'].isin(ids)]
                row.off_df.loc[:, 'sumexp'] = exp_levels
                exp_values.append(sum(exp_levels))
            self.probedf['sumexp'] = exp_values
    def probeCountPerOfftarget(self, df, colname='offtarget'):
        """
        
        """
        offtarget_names = []
        for ix, row in df.iterrows():
            new_items = list(row.off_df[colname].values)
            offtarget_names += new_items
        c = Counter(offtarget_names)
        return c
            
        

In [29]:
from __future__ import division, print_function
from collections import defaultdict
import numpy as np
import pandas as pd



import pyensembl
# from interval import interval

import Bio.SeqIO
from pyfaidx import Fasta


class GencodeGTF():
    def __init__(self, gtfpath, fastapth):
        self.df = pyensembl.GTF(gtfpath).dataframe()
#         self._refseq = self._load_reference_fasta(fastapth)
        self._parse_gene_symbol_from_transcript_name()
#         self._trim_version_numbers()
    def _load_reference_fasta(self, fastapth):
        return Fasta(fastapth)
    def _parse_gene_symbol_from_transcript_name(self):
        symbol_list = []
        for name in self.df['transcript_name'].values:
            try:
                gene_symbol = name.split('-')
                if len(gene_symbol)==2:
                    gene_symbol = gene_symbol[0]
            except:
                gene_symbol = np.nan
            symbol_list.append(gene_symbol)
        self.df['symbol'] = symbol_list
    def _trim_version_numbers(self, trim_list = ['gene_id',
                                                 'transcript_id']):
        new_columns = [[] for i in range(len(trim_list))]
        for idx, row in self.df.iterrows():
            for list_idx, column in enumerate(trim_list):
                try:
                    name = row[column].split('.')
                    if len(name)==2:
                        name = name[0]
                except:
                    name = np.nan
                new_columns[list_idx].append(name)
        for idx, column in enumerate(trim_list):
            self.df[column] = new_columns[idx]
    def ref_lookup(self, seqname, start, end, strand='+'):
        seq = self._refseq[seqname][start:end]
        if strand=='-':
            seq = seq.reverse.complement
        return seq.seq
    def find_isoform_ids(self, tid):
        genename = self.df[self.df['transcript_id']==tid]['gene_id'].iloc[0]
        x = self.df[self.df['gene_id'] == genename]['transcript_id'].unique()
        x = [i for i in x if isinstance(i, str)]
        return x


    



# gtf = GencodeGTF('/Users/rfor10/Downloads/gencode.v24.basic.annotation.gtf', 
#           '/Users/rfor10/Downloads/GRCh38.primary_assembly.genome.fa')
# not_expressed = gtf.df[gtf.df.transcript_support_level!=1].transcript_id.unique()



In [30]:
gtf2 = GencodeGTF('/Users/rfor10/Desktop/Homo_sapiens.GRCh38.84.gtf', 
          '/Users/rfor10/Downloads/GRCh38.primary_assembly.genome.fa')

Reading Dataframe from /Users/rfor10/Desktop/Homo_sapiens.GRCh38.84.expanded.csv


/Users/rfor10/anaconda/lib/python3.5/site-packages/pyensembl/gtf.py:202: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  full_df = self._load_full_dataframe_cached()


In [31]:
gtf2.df

,seqname,source,feature,start,end,score,strand,frame,gene_id,gene_version,...,havana_transcript_version,tag,transcript_support_level,exon_number,exon_id,exon_version,ccds_id,protein_id,protein_version,symbol
0,1,havana,gene,11869,14409,NaN,+,.,ENSG00000223972,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,havana,transcript,11869,14409,NaN,+,.,ENSG00000223972,5,...,1.0,basic,1.0,NaN,NaN,NaN,NaN,NaN,NaN,DDX11L1
2,1,havana,exon,11869,12227,NaN,+,.,ENSG00000223972,5,...,1.0,basic,1.0,1.0,ENSE00002234944,1.0,NaN,NaN,NaN,DDX11L1
3,1,havana,exon,12613,12721,NaN,+,.,ENSG00000223972,5,...,1.0,basic,1.0,2.0,ENSE00003582793,1.0,NaN,NaN,NaN,DDX11L1
4,1,havana,exon,13221,14409,NaN,+,.,ENSG00000223972,5,...,1.0,basic,1.0,3.0,ENSE00002312635,1.0,NaN,NaN,NaN,DDX11L1
5,1,havana,transcript,12010,13670,NaN,+,.,ENSG00000223972,5,...,2.0,basic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DDX11L1
6,1,havana,exon,12010,12057,NaN,+,.,ENSG00000223972,5,...,2.0,basic,NaN,1.0,ENSE00001948541,1.0,NaN,NaN,NaN,DDX11L1
7,1,havana,exon,12179,12227,NaN,+,.,ENSG00000223972,5,...,2.0,basic,NaN,2.0,ENSE00001671638,2.0,NaN,NaN,NaN,DDX11L1
8,1,havana,exon,12613,12697,NaN,+,.,ENSG00000223972,5,...,2.0,basic,NaN,3.0,ENSE00001758273,2.0,NaN,NaN,NaN,DDX11L1
9,1,havana,exon,12975,13052,NaN,+,.,ENSG00000223972,5,...,2.0,basic,NaN,4.0,ENSE00001799933,2.0,NaN,NaN,NaN,DDX11L1


In [16]:
newdf['transcript'] = [t.split('.')[0] for t in newdf['transcript_id'].values if isinstance]

AttributeError: 'float' object has no attribute 'split'

In [32]:
from collections import defaultdict
gene_name_to_transcripts = defaultdict(list)
transcript_to_gene = defaultdict(list)
newdf = gtf2.df[['gene_id', 'transcript_id', 'transcript_name', 'gene_name']].drop_duplicates(subset='transcript_id')
for idx, row in newdf.iterrows():
    if idx % 1000 == 0:
        print(idx)
    gene_id, transcript_id, tname, gname = row
    if isinstance(tname, float):
        continue
    gene_id = gene_id.split('.')[0]
    gene_symbol = gname
    transcript_id = transcript_id.split('.')[0]
    if transcript_id not in gene_name_to_transcripts[gene_id]:
        gene_name_to_transcripts[gene_id].append(transcript_id)
        gene_name_to_transcripts[gene_symbol].append(transcript_id)
    
    transcript_to_gene[transcript_id] =((gene_symbol, gene_id))

0
11000
44000
71000
82000
90000
112000
115000
118000
168000
170000
206000
214000
215000
257000
260000
262000
270000
290000
304000
319000
326000
330000
342000
384000
393000
400000
416000
425000
436000
440000
455000
473000
491000
495000
498000
504000
515000
516000
543000
557000
563000
575000
577000
590000
603000
607000
614000
617000
669000
673000
699000
701000
725000
726000
733000
738000
752000
755000
770000
794000
802000
819000
839000
844000
867000
884000
890000
891000
907000
929000
936000
944000
949000
974000
982000
989000
991000
1021000
1027000
1033000
1046000
1049000
1057000
1080000
1085000
1089000
1094000
1097000
1107000
1109000
1119000
1138000
1143000
1150000
1154000
1164000
1165000
1171000
1178000
1185000
1211000
1219000
1226000
1248000
1269000
1277000
1285000
1290000
1292000
1312000
1313000
1320000
1330000
1358000
1360000
1368000
1377000
1384000
1404000
1414000
1423000
1429000
1430000
1456000
1522000
1523000
1539000
1549000
1551000
1567000
1583000
1621000
1636000
1640000
1645000


In [33]:
import pickle

pickle.dump(gene_name_to_transcripts, open('/data/wollman/hypefish/datafiles/gene2transcripts.pck', 'wb'))
pickle.dump(transcript_to_gene, open('/data/wollman/hypefish/datafiles/transcripts2genes.pck', 'wb'))

In [17]:
if __name__ == '__main__':
    d = DesignTargets()
    # ENST00000356805.8 is the tid of SPTBN1 gene
    d.designProbes(transcript_id=['tdEOS'])
    on = d._parseOligoArrayOutput()
#     targets = ScreenTargets(on)

[SeqRecord(seq=Seq('ATGAGTGCGATTAAGCCAGACATGAAGATCAACCTCCGTATGGAAGGCAACGTA...TAA', SingleLetterAlphabet()), id='tdEOS', name='tdEOS', description='tdEOS', dbxrefs=[])]

	***	OligoArray 2.1.3	***

OligoArray 2.1.3 will start to process sequences from the file /data/a28cedaf-ce2f-4acb-a05e-d40c3e2a1a8c/temp.fa using the following parameters :
Blast database: '/data/a28cedaf-ce2f-4acb-a05e-d40c3e2a1a8c/db.fa'
Oligo data will be saved in: '/data/a28cedaf-ce2f-4acb-a05e-d40c3e2a1a8c/oligos.txt'
Sequence without oligo will be saved in: 'rejected.fas'
The log file will be: '/data/a28cedaf-ce2f-4acb-a05e-d40c3e2a1a8c/oligos.log'
Maximum number of oligo to design per input sequence: '196'
Size range: '30' to '30'
Maximum distance between the 5' end of the oligo and the 3' end of the input sequence: '10000'
Minimum distance between the 5' ends of two adjacent oligos: '30'
Tm range: '70' to '100'
GC range: '20.0' to '80.0'
Threshold to reject secondary structures: '76.0'
Threshold to start to con

In [18]:
on

,target,pos,len,dG,dH,dS,Tm,seq,off_df
0,tdEOS,1486,30,-179.59,-226.39,-615.89,77.25,AGAAAGGTAGGATCCACCGGATCTAGATAA,"Empty DataFrame Columns: [target, pos, offtarg..."
1,tdEOS,1398,30,-190.74,-239.59,-642.8,82.86,ACGATCCGGACTCAGATCTCGAGCTGATCC,"Empty DataFrame Columns: [target, pos, offtarg..."
2,tdEOS,1368,30,-188.34,-236.70,-636.19,82.03,TGCTCATTCTGGATTGCCTGACAATGCCAG,"Empty DataFrame Columns: [target, pos, offtarg..."
3,tdEOS,1338,30,-187.08,-235.7,-639.69,78.68,CAACAAGGTTAAGCTGTATGAGCATGCTGT,"Empty DataFrame Columns: [target, pos, offtarg..."
4,tdEOS,1307,30,-183.19,-231.5,-635.59,74.54,GCATTGAGATTTTAAGCCATGACAAAGATT,"Empty DataFrame Columns: [target, pos, offtarg..."
5,tdEOS,1277,30,-184.49,-231.8,-622.39,82.03,AGTTACCAGGCTACCACTTTGTGGACCACT,"Empty DataFrame Columns: [target, pos, offtarg..."
6,tdEOS,1247,30,-182.99,-230.8,-628.99,76.96,CTACTTACAAAGCTAAGGAGAAGGGTGTCA,"Empty DataFrame Columns: [target, pos, offtarg..."
7,tdEOS,1217,30,-186.17,-234.6,-637.19,78.35,GAAATGCCCATTACCGATGTGACTTCAGAA,"Empty DataFrame Columns: [target, pos, offtarg..."
8,tdEOS,1187,30,-190.23,-239.80,-652.19,78.25,GTGATATTCGCATGGCTTTGTTGCTTGAAG,"Empty DataFrame Columns: [target, pos, offtarg..."
9,tdEOS,1157,30,-189.40,-238.40,-644.69,80.08,AAATGTATGTGCGTGATGGAGTGCTGACTG,"Empty DataFrame Columns: [target, pos, offtarg..."


In [5]:
alt_map = {'ins':'0'}
complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'} 

def reverse_complement(seq):    
    for k,v in alt_map.iteritems():
        seq = seq.replace(k,v)
    bases = list(seq) 
    bases = reversed([complement.get(base,base) for base in bases])
    bases = ''.join(bases)
    for k,v in alt_map.iteritems():
        bases = bases.replace(v,k)
    return bases

# print([reverse_complement(i) for i in on.seq.values])

In [52]:
import cPickle as pickle
exp_df = pickle.load(open('/data/wollman/hypefish/ipython/research-python/imr90_combined_df.pck', 'r'))

In [89]:
import numpy as np

def aggOfftargetExpression(off_df, on_df, expression_df, linker='id',
                           expression_column='SUM_FPKM', aggfunc = sum, arg = None):
    """
    Given a groupby objected formed on the linker column name and 
    an expression_df with the column expression_column. Produce 
    an output where group id's looked up in expression_df and 
    values are aggregated by aggfunc.
    """
    if arg is None:
        arg = {'tm_thresh': 70, 
               }
    on_df['id'] = [row.target+'_'+row.pos for ix, row in on_df.iterrows()]
    on_df['offexp'] = [[] for i in range(on_df.shape[0])]
    off_df['id'] = [row.target+'_'+row.pos for ix, row in off_df.iterrows()]
    off_df = off_df[off_df['tm'].astype(float)>arg['tm_thresh']]
    probe_groups = off_df.groupby('id')
    
    counter = 0
    for key, values in probe_groups.groups.iteritems():
        number_offtargets_this_probe = len(values)
        exp_values = expression_df[expression_column].ix[values]
        agg_value = aggfunc(exp_values)
        if len(exp_values)<1:
            continue
        subset = on_df[on_df['id']==key].index
        on_df.loc[subset, 'offexp'] = [zip(off_df.loc[values, 'offtarget'].values,
                                               exp_values.values, 
                                               off_df.loc[values, 'tm'].astype(float).values)]
    return on_df

